In [1]:
import torch
import torch.nn as nn
from torchvision.models import resnet18
from torchvision import transforms
from PIL import Image

# 定义常量
NUM_CLASSES = 10

# 初始化ResNet-18模型
model = resnet18(pretrained=False)  # 此处不加载预训练权重
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)  # 修改分类器的输出层

# 加载保存的模型参数并将模型移到GPU上
model.load_state_dict(torch.load('pretrained_resnet_animal_classifier.pth'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()  # 设置为评估模式

# 图片路径
image_path = 'cat.jpg'

# 预处理图片
input_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 读取图片并进行预处理
img = Image.open(image_path).convert('RGB')
input_tensor = input_transform(img)
input_batch = input_tensor.unsqueeze(0).to(device)  # 添加批次维度并移到GPU上

# 使用模型进行预测
with torch.no_grad():
    output = model(input_batch)

# 获取预测结果
_, predicted_idx = torch.max(output, 1)
predicted_label = predicted_idx.item()

# 映射预测结果到食物
food_mapping = {
    0: "狗粮",
    1: "青甘草",
    2: "树皮",
    3: "花蜜",
    4: "熟米",
    5: "猫粮",
    6: "胡萝卜",
    7: "菜叶",
    8: "虫子",
    9: "坚果"
}

predicted_food = food_mapping.get(predicted_label, "未知食物")
print(f'应该投喂: {predicted_food}')


/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.